# Chiral solutions for Type-I Dirac seesaw

In [3]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)
pd.set_option('display.max_columns',100)

## Load full solutions

In [4]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(148097, 5)

Check that solutions are unique

In [5]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(148097, 6)

In [6]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [7]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

95358

## Check phenomenological conditions
$$ 2l+\nu-m=0$$

In [8]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [9]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    for x in ll:
        if ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level(ll):
    '''
    Given a solution find set of repeated charges and assign to `l` and `ν`, e.g
       ll=[l,l,...,ν,ν,...]
    For each combination of repeated sets calculates:
       m=2l+ν
    If `m ≠ l ≠ ν` in ll, returns a `N → N+4` solution
       [m,m,m,-m-m,l,l,...,ν,ν,...]
    else, returns a `N → N+6` solution
       [m,m,m,-m,-m,-m,l,l,...,ν,ν,...]
    '''
    ms=[]
    rp=[]
    for x in ll:
        if ll.count(x)==1:
            ms.append(x)
        elif ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    #print(rp,isinstance(rp,list))
    #Two or more repeated sets
    sltn=[]
    if len(rp)>=2:
        rps=[k  for k  in itertools.combinations(rp,2)]
        #print('rps1',rps)
        rps=rps+[tuple(reversed(k)) for k  in rps]
        #print('rps2',rps)
        for r in rps:
            #Loop on combinatories here, break whit the first solution
            l=r[0]
            ν=r[1]
            m=2*l+ν
            s=l-m
            massless=ms.copy()
            massless=massless+[x for x in rp if x not in r and x not in massless]
            #print('m,s',massless,s,m,rp)
            #print('ini massless',massless)
            if m in ll and m not in rp:
                massless.remove(m)
                #Check Dirac and Majorana
                massless=extract_Dirac_and_Majorana(massless,s)
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,m,0],'massless':massless}
            else:
                massless=extract_Dirac_and_Majorana(massless,s)                
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,0,m],'massless':massless}
                
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)
            sltn.append(slt)
    return sltn

assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
assert best_higgs_singlet([1,-11,9])==(2, [])
assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None
assert not tree_level([1, 1, -3, -4, 6, 6, -7])[0]['sp_massless']

### Initialize filtered solutions

In [10]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].reset_index(drop=True)

Find solutions that satisfy the condition

In [11]:
cl['tree_level']=cl['solution'].apply(tree_level)

Reorder the filtered solutions

In [12]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
             )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5


In [13]:
cl.shape

(6385, 7)

### Flag equivalent solutions

In [14]:
def characterize_solution(cl,i,j=0):
    if j%2==0:
        cl.loc[i,'D_msls']=j
    else:
        cl.loc[i,'M_msls']=j
    #if j==0:
    #    cl.loc[i,'M_msls']=j
        
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    sl=[d.get('[l,ν,s,m_in,m_out]') for d in cl.loc[i,'tree_level'] if len(d.get('massless'))==j]
    if len(sl)>0:
        cl.loc[i,'nl']=cl.loc[i,'solution'].count(sl[0][0] )
        cl.loc[i,'nν'] =cl.loc[i,'solution'].count(sl[0][1] )
    if 0 in [l[3] for l in sl]:
        cl.loc[i,'nr']=3
    else:
        cl.loc[i,'nr']=2
    return cl

In [15]:
for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
    cl=characterize_solution(cl,i,0)

Example: massless solutions

In [16]:
cl.dropna()[:5]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr
20,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,"[{'[l,ν,s,m_in,m_out]': [-7, -2, 9, 0, -16], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [-2, -7, 9, 0, -11], 'massless': [], 'sp': None, 'sp_massless': None}]",8,0.0,2.0,2.0,2.0,3.0
22,"[2, 0, -1]","[-8, -5, 9]","[2, -3, -3, 5, 6, -8, -8, 9]",1,8,"[{'[l,ν,s,m_in,m_out]': [-8, -3, 11, 0, -19], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [-3, -8, 11, 0, -14], 'massless': [], 'sp': None, 'sp_massless': None}]",9,0.0,2.0,2.0,2.0,3.0
23,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,8,"[{'[l,ν,s,m_in,m_out]': [-6, 7, -1, 0, -5], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [7, -6, -1, 8, 0], 'massless': [-9], 'sp': 18, 'sp_massless': []}]",9,0.0,2.0,2.0,2.0,3.0
27,"[-3, -5, 1]","[-6, -4, -5]","[3, -4, -4, 6, 7, -9, -9, 10]",1,8,"[{'[l,ν,s,m_in,m_out]': [-9, -4, 13, 0, -22], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [-4, -9, 13, 0, -17], 'massless': [], 'sp': None, 'sp_massless': None}]",10,0.0,2.0,2.0,2.0,3.0
28,"[1, -1, 1]","[-2, -1, -2]","[1, -2, -7, -7, 8, 8, 10, -11]",1,8,"[{'[l,ν,s,m_in,m_out]': [-7, 8, -1, 0, -6], 'massless': [], 'sp': None, 'sp_massless': None}, {'[l,ν,s,m_in,m_out]': [8, -7, -1, 0, 9], 'massless': [], 'sp': None, 'sp_massless': None}]",11,0.0,2.0,2.0,2.0,3.0


### Add solutions with massless fermions that get masses with some $S'$

In [17]:
for j in range(1,6):
    print(j)
    for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if d.get('sp_massless') is not None and len(d.get('sp_massless'))==0  and len(d.get('massless'))==j ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
        cl=characterize_solution(cl,i,j)

1
2
3
4
5


In [18]:
cl=cl[cl['n']<10].reset_index(drop=True)

In [19]:
cl['D_msls']=cl['D_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['M_msls']=cl['M_msls'].apply(lambda n: 10 if np.isnan(n) else n)
for k  in ['D_msls','M_msls','nl','nν','nr','rptd']:
    cl[k]=cl[k].astype(int)

In [20]:
cl=cl.sort_values(['n','smax','D_msls','rptd','nl','nν','nr']).reset_index(drop=True)

In [21]:
cl['sltn']=cl['solution'].astype(str)
ds['sltn']=ds['solution'].astype(str)

## Build the final table dropping out equivalent solutions 

In [22]:
fl=pd.DataFrame()

#fl=fl.append( cl[( (cl['n']==9) & (cl['D_msls']==0) )].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )
fl=fl.append( cl[ cl['n']==9 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )
fl=fl.append( cl[ cl['n']==6 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )
fl=fl.append(cl[ cl['n']==7 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )
fl=fl.append(cl[ cl['n']==8 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )
fl=fl.reset_index(drop=True)

In [23]:
def getit(it,dk):
    it['ll']=dk.get('[l,ν,s,m_in,m_out]')[0]
    it['ν']=dk.get('[l,ν,s,m_in,m_out]')[1]
    it['s']=dk.get('[l,ν,s,m_in,m_out]')[2]
    it['massless']=dk.get('massless')
    l=it['solution'].copy()    
    nν=l.count(it['ν'])
    ll=l.count(it['ll'])
    if dk.get('[l,ν,s,m_in,m_out]')[3]!=0:
        m=dk.get('[l,ν,s,m_in,m_out]')[3]
        if ll==2:#True:#nν==2:# and ll==2:
            it['extra']=[m,m,-m,-m]
        else:
            it['extra']=None
    else:
        m=dk.get('[l,ν,s,m_in,m_out]')[4]
        if ll==3:
            it['extra']=[m,m,m,-m,-m,-m]
        else:
            it['extra']=None            
    it['sp']=dk.get('sp')
    return it
    
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax','D_msls','M_msls']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    if fl.loc[i,'D_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:
                it['nn']=it['n']+len(it['extra'])
            tm=tm.append(it,ignore_index=True)
    elif fl.loc[i,'M_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)
    else:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:            
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            if it['extra']:            
                it['nn']=it['n']+len(it['extra'])            
            tm=tm.append(it,ignore_index=True)

In [24]:
tm[tm['solution'].astype(str)=='[1, 1, 2, 2, 4, -5, -7, -7, 9]']

,D_msls,M_msls,extra,gcd,k,l,ll,massless,n,nl,nr,nν,rptd,s,sltn,smax,solution,sp,tree_level,ν,nn
37,2.0,1.0,"[-5, -5, 5, 5]",1.0,"[-5, -3, -6, -3]","[1, 2, 1, 2]",1.0,[9],9.0,2.0,3.0,2.0,3.0,6.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",9.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",18,"[{'[l,ν,s,m_in,m_out]': [-7, 1, 6, 0, -13], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-7, 2, 5, 0, -12], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -7, 6, -5, 0], 'massless': [9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -7, 5, 0, -3], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, 1...",-7.0,13.0
38,2.0,1.0,None,1.0,"[-5, -3, -6, -3]","[1, 2, 1, 2]",-7.0,"[-5, 9]",9.0,2.0,3.0,2.0,3.0,6.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",9.0,"[1, 1, 2, 2, 4, -5, -7, -7, 9]",4,"[{'[l,ν,s,m_in,m_out]': [-7, 1, 6, 0, -13], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-7, 2, 5, 0, -12], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -7, 6, -5, 0], 'massless': [9], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -7, 5, 0, -3], 'massless': [-5, 9], 'sp': 4, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, 1...",1.0,13.0


In [25]:
tm=tm.dropna(subset=['extra']).reset_index(drop=True)

In [26]:
tm.shape

(37, 21)

In [27]:
tm['nnn']=(tm['extra'].apply(len)/2).astype(int)+tm['n']

In [28]:
for k in ['nnn','gcd','ll','ν','s','n','D_msls','rptd','nl','nν','nr','M_msls']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs(row['extra']+row['solution']).max(),axis='columns' )
tm['lextra']=tm['extra'].apply(len)
tm=tm.sort_values(['nnn','n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
tm['Q']=True
sltns={#'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],
       #'[1, -4, -9]':[1, -4, 5, 5, -9, -9, -9, 10, 10],
       #'[1, -5, 8, -17, 21]':[1, -5, 8, 12, 12, -16, -16, -17, 21],
       # '[25, 29]': [9, 9, -24, -24, -24, 25, 29],
       # '[13, 23]':[5, 5, 5, 13, -17, -17, -17, 23]#,
       #'[-4, 6, -10, 12]':[3, 3, -4, 6, -10, -11, -11, 12, 12]       
      }
for k in sltns.keys():
    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
          )].index[0] #unstable dark matter
    tm.loc[iq,'Q']=False
#alltm=tm.copy()
tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['nnn','n','l','k','extra','solution','gcd','ll','ν','s','massless','sp','M_msls','rptd','nl','nν','nr','D_msls']
  ].sort_values(['nnn','M_msls','rptd','nl','nν','nr'])#.loc[[0,4]]#.loc[[0,4]]

,nnn,n,l,k,extra,solution,gcd,ll,ν,s,massless,sp,M_msls,rptd,nl,nν,nr,D_msls
0,8,6,"[1, -2]","[-4, 1]","[1, 1, -1, -1]","[1, -4, -4, 9, 9, -11]",3,-4,9,-5,[-11],22,1,2,2,2,3,2
1,9,7,"[-1, 1]","[-1, 0, -1]","[1, 1, -1, -1]","[1, 2, 2, -3, -3, -3, 4]",1,2,-3,1,[4],8,1,2,3,2,3,2
2,9,7,"[3, 1]","[-1, -5, 7]","[-4, -4, 4, 4]","[2, 2, -4, 7, -8, -8, 9]",1,2,-8,6,"[7, 9]",16,10,2,2,2,2,2
5,10,8,"[0, -3, -1]","[-3, -5, -6, -4]","[8, 8, -8, -8]","[3, -4, -6, -6, 7, 7, 8, -9]",1,7,-6,-1,[-9],18,1,2,2,2,2,0
6,10,8,"[-4, -1, 1]","[-13, -14, -7]","[-2, -2, 2, 2]","[1, 1, -2, -4, -4, 10, 11, -13]",2,1,-4,3,[11],22,1,2,2,2,3,2
3,10,7,"[-1, 1]","[-1, 0, -1]","[-4, -4, -4, 4, 4, 4]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,1,"[1, 4]",5,1,2,3,2,3,2
7,10,8,"[0, 2, 3]","[-1, -7, -4, -5]","[1, 1, -1, -1]","[1, -5, -5, -10, 11, 11, 11, -14]",2,-5,11,-6,"[-10, -14]",24,10,2,2,3,2,2
4,10,7,"[9, -6]","[-5, -10, 8]","[-39, -39, -39, 39, 39, 39]","[9, 9, -24, -24, -24, 25, 29]",45,-24,9,15,"[25, 29]",54,10,2,3,2,3,2
8,10,8,"[0, -2, -3]","[-1, -9, -6, -4]","[17, 17, -17, -17]","[3, 3, 7, 7, -12, -12, -13, 17]",1,7,3,-10,"[-13, -12]",25,10,3,2,2,2,2
21,11,9,"[-1, 2, 3, 2]","[-1, -3, -2, -5]","[-9, -9, 9, 9]","[1, 1, 2, 5, 8, -9, -11, -11, 14]",1,1,-11,10,[14],28,1,2,2,2,3,2


In [29]:
def rank(l,massless,s):
    '''
    Return 0 if the integers in massless appears as s-paired same-dimension multiplets in l, 1 otherwise.
    s-paired means that exist two intengers in massless such that n1+n2=±s. (n1 can be equal to n2)
    
    For example, for l containing only the massless=[1,2] integers:
        l=[1,2]; s=3 → 0: 3-paired multiplet: ([1],[2]) of 1-dimension multiplets (A.K.A singlet)
        l=[1,1,2]; s=3 → 1: unpaired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2]; s=3 → 0: 3-paired multiplets: ([1,1],[2]) doblet and singlet
        l=[1,1,2,2,2]; s=3 → 1: unpaired multiplets: ([1,1,1],[2,2]) doblet and singlet
    massless=[1,2,3]
    l=[1,2,3]; s=4 → 0: 4-paired multiplet: ([1],[3]) of singlets
                            4-paired multiplet: ([1],[3]) of singlet
    '''
    #Keep massless
    l=[x for x in  l if x in massless ]
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    #Get s-multiplets
    xxx=[l.count(x) for x in xx if l.count(x)!=1] 
    if len(xxx)%2==0: # s-multiplets must be paired
        # Check that each mutiplet is paired
        if 1 in [xxx.count(x)%2 for x in set(xxx)]:
            return 1
        else:
            #list of only 0's → if dimension of multiplets are paired
            return 0 # 
    else:
        return 1
        
assert rank([1,2],[1,2],3)==0
assert rank([1,1,2,2,2],[1,2],3)==1
assert rank([1,2,3],[1,2,3],4)==0
assert rank([1, 1, -4, 9, -10, -10, 16, 16, -19],[9, -19, 1],18)==1
assert rank( [1, -4, -4, 9, 9, -11], [-11],22)==0
assert rank( [-16,-16,-16, 17, 17],[-16, 17],1)==1
assert rank( [-16,-16,-16, 17, 17,-18,-18,19,19,19],[-16, 17,-18,19],1)==0

In [30]:
tm=tm[tm.apply(lambda row: rank(row['solution'],row['massless'],row['sp']) if row['sp'] else 0,axis='columns')==0].reset_index(drop=True)

In [31]:
#assert tm[tm['sltn']=='[1, 1, -3, -4, 6, 6, -7]'].iloc[-1]['massless']==[-7]

In [32]:
kk=tm[['nn','l','k','solution','gcd','extra','ll','ν','s','massless','sp']].copy()

In [33]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [34]:
kk=tm[['nnn','l','k','solution','gcd','extra','ll','ν','s','massless','sp']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'nnn']=add_boldsymbol(kk.loc[i,'nnn'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

20
26


In [35]:
cat solutions.tex

\begin{tabular}{llllrlrrrll}
\toprule
             nnn &               l &                 k &                                solution & gcd &                        extra &  ll &   ν &  s &        massless &    sp \\
\midrule
               8 &         (1, -2) &           (-4, 1) &                  (1, -4, -4, 9, 9, -11) &   3 &               (1, 1, -1, -1) &  -4 &   9 & -5 &           (-11) &    22 \\
               9 &         (-1, 1) &       (-1, 0, -1) &                (1, 2, 2, -3, -3, -3, 4) &   1 &               (1, 1, -1, -1) &   2 &  -3 &  1 &             (4) &     8 \\
               9 &          (3, 1) &       (-1, -5, 7) &                (2, 2, -4, 7, -8, -8, 9) &   1 &               (-4, -4, 4, 4) &   2 &  -8 &  6 &          (7, 9) &    16 \\
              10 &         (-1, 1) &       (-1, 0, -1) &                (1, 2, 2, -3, -3, -3, 4) &   1 &        (-4, -4, -4, 4, 4, 4) &  -3 &   2 &  1 &          (1, 4) &     5 \\
              10 &         (9, -6) &      (-5, -10, 8

In [36]:
kkk[kkk['solution'].astype(str)=='[1, 1, 2, 2, 4, -5, -7, -7, 9]']

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp
12,11,"[1, 2, 1, 2]","[-5, -3, -6, -3]","[1, 1, 2, 2, 4, -5, -7, -7, 9]",1,"[-5, -5, 5, 5]",1,-7,6,[9],18


In [37]:
kkk

,nnn,l,k,solution,gcd,extra,ll,ν,s,massless,sp
0,8,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,"[1, 1, -1, -1]",-4,9,-5,[-11],22
1,9,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,"[1, 1, -1, -1]",2,-3,1,[4],8
2,9,"[3, 1]","[-1, -5, 7]","[2, 2, -4, 7, -8, -8, 9]",1,"[-4, -4, 4, 4]",2,-8,6,"[7, 9]",16
3,10,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,"[-4, -4, -4, 4, 4, 4]",-3,2,1,"[1, 4]",5
4,10,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,"[-39, -39, -39, 39, 39, 39]",-24,9,15,"[25, 29]",54
5,10,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,"[8, 8, -8, -8]",7,-6,-1,[-9],18
6,10,"[-4, -1, 1]","[-13, -14, -7]","[1, 1, -2, -4, -4, 10, 11, -13]",2,"[-2, -2, 2, 2]",1,-4,3,[11],22
7,10,"[0, 2, 3]","[-1, -7, -4, -5]","[1, -5, -5, -10, 11, 11, 11, -14]",2,"[1, 1, -1, -1]",-5,11,-6,"[-10, -14]",24
8,11,"[0, 1, -2]","[-1, -2, -4, 1]","[2, 2, 2, 2, -5, -5, -5, 7]",2,"[-8, -8, -8, 8, 8, 8]",-5,2,3,[7],14
9,11,"[-1, 0, -1]","[-2, 1, -1]","[2, -5, -5, -5, 7, 8, 8, -10]",1,"[-2, -2, -2, 2, 2, 2]",-5,8,-3,[2],4


In [143]:
pd.set_option('display.max_rows', 500)

In [144]:
tm[['nn','l','k','extra','solution','gcd','ll','ν','s','massless','sp','D_msls','rptd','nl','nν','nr','M_msls']].drop_duplicates(subset=['nn','D_msls','rptd','nl','nν','nr','M_msls'])

,nn,l,k,extra,solution,gcd,ll,ν,s,massless,sp,D_msls,rptd,nl,nν,nr,M_msls
0,10,"[1, -2]","[-4, 1]","[1, 1, -1, -1]","[1, -4, -4, 9, 9, -11]",3,-4,9,-5,[-11],22,2,2,2,2,3,1
1,11,"[-1, 1]","[-1, 0, -1]","[1, 1, -1, -1]","[1, 2, 2, -3, -3, -3, 4]",1,2,-3,1,[4],8,2,2,3,2,3,1
2,11,"[3, 1]","[-1, -5, 7]","[-4, -4, 4, 4]","[2, 2, -4, 7, -8, -8, 9]",1,2,-8,6,"[7, 9]",16,2,2,2,2,2,10
3,12,"[-1, -2]","[-1, 2]","[-7, -7, -7, 7, 7, 7]","[1, 1, 1, -4, -4, 5]",1,-4,1,3,[5],10,10,2,2,3,3,1
4,12,"[1, -2]","[-4, 1]","[14, 14, 14, -14, -14, -14]","[1, -4, -4, 9, 9, -11]",3,9,-4,-5,"[1, -11]",10,2,2,2,2,3,1
5,12,"[0, 1]","[-1, -3, 1]","[-17, -17, -17, 17, 17, 17]","[1, 1, 5, -9, -9, 11]",2,-9,1,8,"[5, 11]",16,2,2,2,2,3,10
6,12,"[0, -3, -1]","[-3, -5, -6, -4]","[8, 8, -8, -8]","[3, -4, -6, -6, 7, 7, 8, -9]",1,7,-6,-1,[-9],18,0,2,2,2,2,1
8,12,"[0, 2, 3]","[-1, -7, -4, -5]","[1, 1, -1, -1]","[1, -5, -5, -10, 11, 11, 11, -14]",2,-5,11,-6,"[-10, -14]",24,2,2,2,3,2,10
9,12,"[0, -2, -3]","[-1, -9, -6, -4]","[17, 17, -17, -17]","[3, 3, 7, 7, -12, -12, -13, 17]",1,7,3,-10,"[-13, -12]",25,2,3,2,2,2,10
10,12,"[-1, -2, 2]","[-10, -1, -8]","[1, 1, -1, -1]","[1, 10, 10, 10, -19, -19, -19, 26]",17,10,-19,9,[26],52,2,2,3,3,3,1


Matrix rank

In [145]:
import scipy as sp
import numpy as np
#For arbitrary precision see:
#https://stackoverflow.com/questions/6876377/numpy-arbitrary-precision-linear-algebra

In [146]:
n=3
Z=np.zeros((n,n))
MD=np.random.uniform(100,200,(n,n))
MS=np.random.uniform(100,200,(n,n))
MF=np.zeros((n,n))
if n>=1:
    MF[0,0]=1E5
if n>=2:
    MF[1,1]=1E6
if n>=3:
    MF[2,2]=1E7

In [115]:
#M=np.vstack( (np.hstack((Z,MD)),np.hstack((MS,MF))) )
M=np.vstack( (np.hstack((Z,MD)),np.hstack((MD.transpose(),MF))) )
#M=np.longdouble(M)
M.round(1)

array([[0.000e+00, 0.000e+00, 0.000e+00, 1.543e+02, 1.655e+02, 1.344e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, 1.843e+02, 1.508e+02, 1.690e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, 1.694e+02, 1.012e+02, 1.103e+02],
       [1.543e+02, 1.843e+02, 1.694e+02, 1.000e+05, 0.000e+00, 0.000e+00],
       [1.655e+02, 1.508e+02, 1.012e+02, 0.000e+00, 1.000e+06, 0.000e+00],
       [1.344e+02, 1.690e+02, 1.103e+02, 0.000e+00, 0.000e+00, 1.000e+07]])

In [38]:
#sp.linalg.eig(M)
np.linalg.eig(M)

(array([ 1.00000000e+07,  1.00000004e+06,  1.00000719e+05, -7.62392984e-01,
        -1.73122818e-03, -8.62756888e-07]),
 array([[-1.19350400e-05,  1.13304032e-04,  1.13156348e-03,
          4.30560288e-01, -7.49489404e-01, -5.02873998e-01],
        [-1.40652990e-05,  1.15052898e-04,  1.46464172e-03,
          5.48833263e-01, -2.24900539e-01,  8.05108441e-01],
        [-1.60045707e-05,  1.13885608e-04,  1.94047447e-03,
          7.16514242e-01,  6.22643862e-01, -3.14512545e-01],
        [-6.58212849e-10,  5.75244349e-08,  9.99996404e-01,
         -2.68143774e-03, -3.07310535e-05,  1.42002560e-07],
        [-5.32581245e-10,  9.99999980e-01, -5.75238334e-07,
         -1.93529584e-04,  3.98854582e-05,  1.66044215e-07],
        [-1.00000000e+00, -5.32581289e-09, -6.58205501e-08,
         -2.43257592e-05,  2.14333161e-06, -2.88631414e-07]]))

In [239]:
(1/137)/(2*np.pi)

0.001161714913079528